In [2]:
from bs4 import BeautifulSoup
import requests

In [3]:
url = "https://books.toscrape.com/"
response = requests.get(url)
response.encoding = response.apparent_encoding

In [4]:
response.text

'<!DOCTYPE html>\n<!--[if lt IE 7]>      <html lang="en-us" class="no-js lt-ie9 lt-ie8 lt-ie7"> <![endif]-->\n<!--[if IE 7]>         <html lang="en-us" class="no-js lt-ie9 lt-ie8"> <![endif]-->\n<!--[if IE 8]>         <html lang="en-us" class="no-js lt-ie9"> <![endif]-->\n<!--[if gt IE 8]><!--> <html lang="en-us" class="no-js"> <!--<![endif]-->\n    <head>\n        <title>\n    All products | Books to Scrape - Sandbox\n</title>\n\n        <meta http-equiv="content-type" content="text/html; charset=UTF-8" />\n        <meta name="created" content="24th Jun 2016 09:29" />\n        <meta name="description" content="" />\n        <meta name="viewport" content="width=device-width" />\n        <meta name="robots" content="NOARCHIVE,NOCACHE" />\n\n        <!-- Le HTML5 shim, for IE6-8 support of HTML elements -->\n        <!--[if lt IE 9]>\n        <script src="//html5shim.googlecode.com/svn/trunk/html5.js"></script>\n        <![endif]-->\n\n        \n            <link rel="shortcut icon" href

In [5]:
soup = BeautifulSoup(response.text, "html5lib")


Titre du premier livre

In [15]:
# 1) Premier livre
book = soup.find("article", class_="product_pod")

title = None
if book:
    # cibler spécifiquement le lien dans le <h3>
    a_h3 = book.select_one("h3 a")  # équiv. book.find("h3").find("a") si <h3> existe
    if a_h3:
        # priorité à l'attribut title (le plus fiable sur ce site)
        title = a_h3.get("title") or a_h3.get_text(strip=True)
    else:
        # fallback ultime : l'alt de l'image si présent
        img = book.select_one(".image_container img")
        if img and img.get("alt"):
            title = img.get("alt")

print("Titre du premier livre :", title)


Titre du premier livre : A Light in the Attic


Tous les titres de la page

In [16]:
titles = []
for a in soup.select("article.product_pod h3 a"):
    titles.append(a.get("title") or a.get_text(strip=True))

print(titles[:10])  # aperçu


['A Light in the Attic', 'Tipping the Velvet', 'Soumission', 'Sharp Objects', 'Sapiens: A Brief History of Humankind', 'The Requiem Red', 'The Dirty Little Secrets of Getting Your Dream Job', 'The Coming Woman: A Novel Based on the Life of the Infamous Feminist, Victoria Woodhull', 'The Boys in the Boat: Nine Americans and Their Epic Quest for Gold at the 1936 Berlin Olympics', 'The Black Maria']


In [17]:
info = soup.find("form", class_="form-horizontal")

if info:
    text_info = info.get_text(strip=True)
    print("Texte brut :", text_info)

    # Exemple de parsing simple
    import re
    match = re.search(r"(\d+)\s+results", text_info)
    if match:
        total_results = int(match.group(1))
        print("Nombre total de résultats :", total_results)
else:
    print("⚠️ Aucun élément <form class='form-horizontal'> trouvé.")


Texte brut : 1000results - showing1to20.


In [18]:
# Trouver tous les articles de livres
books = soup.find_all("article", class_="product_pod")

# Liste pour stocker les livres à 1 étoile
one_star_books = []

# Boucle sur chaque livre
for book in books:
    rating_tag = book.find("p", class_="star-rating")
    
    # Vérifie que la balise existe et que 'One' est dans ses classes CSS
    if rating_tag and "One" in rating_tag.get("class", []):
        # Récupère le titre via la balise <h3><a title="...">
        a_tag = book.select_one("h3 a")
        if a_tag:
            title = a_tag.get("title") or a_tag.get_text(strip=True)
            one_star_books.append(title)

print(f"Nombre de livres notés 1 étoile : {len(one_star_books)}")
print("Exemples :", one_star_books[:5])


Nombre de livres notés 1 étoile : 6
Exemples : ['Tipping the Velvet', 'Soumission', 'The Requiem Red', 'The Black Maria', 'Olio']


In [19]:
# Trouver tous les livres
books = soup.find_all("article", class_="product_pod")

# Créer une liste pour stocker les livres à 4 ou 5 étoiles
high_rated_books = []

for book in books:
    # Récupère la balise qui contient les étoiles
    rating_tag = book.find("p", class_="star-rating")
    
    # Vérifie que la balise existe
    if rating_tag:
        # Récupère la liste de classes CSS (ex: ['star-rating', 'Four'])
        classes = rating_tag.get("class", [])
        
        # Vérifie si 'Four' ou 'Five' est présent dans les classes
        if "Four" in classes or "Five" in classes:
            # Récupère le titre dans <h3><a title="...">
            a_tag = book.select_one("h3 a")
            if a_tag:
                title = a_tag.get("title") or a_tag.get_text(strip=True)
                high_rated_books.append(title)

# Afficher les résultats
print(f"Nombre de livres avec 4 ou 5 étoiles : {len(high_rated_books)}")
print("Exemples :", high_rated_books[:5])


Nombre de livres avec 4 ou 5 étoiles : 8
Exemples : ['Sharp Objects', 'Sapiens: A Brief History of Humankind', 'The Dirty Little Secrets of Getting Your Dream Job', 'The Boys in the Boat: Nine Americans and Their Epic Quest for Gold at the 1936 Berlin Olympics', "Shakespeare's Sonnets"]


In [20]:
import pandas as pd

data = []
for book in books:
    rating_tag = book.find("p", class_="star-rating")
    if rating_tag:
        classes = rating_tag.get("class", [])
        rating = next((c for c in classes if c in ["One","Two","Three","Four","Five"]), None)
        a_tag = book.select_one("h3 a")
        title = a_tag.get("title") if a_tag else None
        data.append({"Titre": title, "Note": rating})

df = pd.DataFrame(data)
df_high = df[df["Note"].isin(["Four", "Five"])]

print(df_high.head())


                                                Titre  Note
3                                       Sharp Objects  Four
4               Sapiens: A Brief History of Humankind  Five
6   The Dirty Little Secrets of Getting Your Dream...  Four
8   The Boys in the Boat: Nine Americans and Their...  Four
11                              Shakespeare's Sonnets  Four


In [21]:
import requests
from bs4 import BeautifulSoup

url = "https://books.toscrape.com/catalogue/page-2.html"

resp = requests.get(url, timeout=10)
resp.raise_for_status()
soup = BeautifulSoup(resp.text, "html5lib")

# Chaque titre est dans <article class="product_pod"><h3><a title="...">
titles = [a.get("title") or a.get_text(strip=True)
          for a in soup.select("article.product_pod h3 a")]

print(f"{len(titles)} titres trouvés")
for t in titles:
    print(t)


20 titres trouvés
In Her Wake
How Music Works
Foolproof Preserving: A Guide to Small Batch Jams, Jellies, Pickles, Condiments, and More: A Foolproof Guide to Making Small Batch Jams, Jellies, Pickles, Condiments, and More
Chase Me (Paris Nights #2)
Black Dust
Birdsong: A Story in Pictures
America's Cradle of Quarterbacks: Western Pennsylvania's Football Factory from Johnny Unitas to Joe Montana
Aladdin and His Wonderful Lamp
Worlds Elsewhere: Journeys Around Shakespeareâs Globe
Wall and Piece
The Four Agreements: A Practical Guide to Personal Freedom
The Five Love Languages: How to Express Heartfelt Commitment to Your Mate
The Elephant Tree
The Bear and the Piano
Sophie's World
Penny Maybe
Maude (1883-1993):She Grew Up with the country
In a Dark, Dark Wood
Behind Closed Doors
You can't bury them all: Poems


In [22]:
import time
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

BASE = "https://books.toscrape.com/"
START = urljoin(BASE, "catalogue/page-1.html")  # point d’entrée catalogue

def fetch_soup(url: str) -> BeautifulSoup | None:
    try:
        r = requests.get(url, timeout=10, headers={"User-Agent": "Mozilla/5.0"})
        r.raise_for_status()
        return BeautifulSoup(r.text, "html5lib")
    except requests.RequestException as e:
        print("⛔️ Requête échouée:", e)
        return None

def parse_books(soup: BeautifulSoup) -> list[dict]:
    out = []
    for art in soup.select("article.product_pod"):
        a = art.select_one("h3 a")
        title = a.get("title") or a.get_text(strip=True) if a else None
        price_tag = art.select_one("p.price_color")
        price = price_tag.get_text(strip=True) if price_tag else None
        out.append({"title": title, "price": price})
    return out

def next_page_url(soup: BeautifulSoup, current_url: str) -> str | None:
    nxt = soup.select_one("li.next a")
    return urljoin(current_url, nxt["href"]) if nxt and nxt.get("href") else None

all_books = []
url = START

while url:
    soup = fetch_soup(url)
    if not soup:
        break
    books = parse_books(soup)
    all_books.extend(books)
    print(f"✅ {len(books)} livres récupérés sur {url}")
    url = next_page_url(soup, url)  # None quand il n’y a plus de page suivante
    time.sleep(0.3)  # petite politesse réseau

print(f"\nTotal livres récupérés : {len(all_books)}")

# (optionnel) Export CSV
# import csv
# with open("books_all.csv", "w", encoding="utf-8", newline="") as f:
#     w = csv.DictWriter(f, fieldnames=["title", "price"])
#     w.writeheader()
#     w.writerows(all_books)


✅ 20 livres récupérés sur https://books.toscrape.com/catalogue/page-1.html
✅ 20 livres récupérés sur https://books.toscrape.com/catalogue/page-2.html
✅ 20 livres récupérés sur https://books.toscrape.com/catalogue/page-3.html
✅ 20 livres récupérés sur https://books.toscrape.com/catalogue/page-4.html
✅ 20 livres récupérés sur https://books.toscrape.com/catalogue/page-5.html
✅ 20 livres récupérés sur https://books.toscrape.com/catalogue/page-6.html
✅ 20 livres récupérés sur https://books.toscrape.com/catalogue/page-7.html
✅ 20 livres récupérés sur https://books.toscrape.com/catalogue/page-8.html
✅ 20 livres récupérés sur https://books.toscrape.com/catalogue/page-9.html
✅ 20 livres récupérés sur https://books.toscrape.com/catalogue/page-10.html
✅ 20 livres récupérés sur https://books.toscrape.com/catalogue/page-11.html
✅ 20 livres récupérés sur https://books.toscrape.com/catalogue/page-12.html
✅ 20 livres récupérés sur https://books.toscrape.com/catalogue/page-13.html
✅ 20 livres récupérés

In [23]:
import requests
from bs4 import BeautifulSoup
import re

BASE = "https://books.toscrape.com/catalogue/"
FIRST = BASE + "page-1.html"

def get_total_pages() -> int:
    r = requests.get(FIRST, timeout=10)
    r.raise_for_status()
    soup = BeautifulSoup(r.text, "html5lib")
    # Sur ce site, l’info est dans: <li class="current">Page 1 of 50</li>
    li = soup.select_one("li.current")
    if not li:
        return 1
    m = re.search(r"Page\s+\d+\s+of\s+(\d+)", li.get_text(strip=True), re.I)
    return int(m.group(1)) if m else 1

def parse_page(n: int) -> list[dict]:
    url = BASE + f"page-{n}.html"
    r = requests.get(url, timeout=10)
    r.raise_for_status()
    soup = BeautifulSoup(r.text, "html5lib")
    rows = []
    for art in soup.select("article.product_pod"):
        a = art.select_one("h3 a")
        title = a.get("title") or a.get_text(strip=True) if a else None
        price = (art.select_one("p.price_color") or {}).get_text(strip=True) if art.select_one("p.price_color") else None
        rows.append({"title": title, "price": price})
    return rows

total_pages = get_total_pages()
print("Pages totales :", total_pages)

all_books = []
for i in range(1, total_pages + 1):
    try:
        page_books = parse_page(i)
        all_books.extend(page_books)
        print(f"✅ Page {i}: {len(page_books)} livres")
    except requests.HTTPError as e:
        print(f"⚠️ Page {i} inaccessible ({e}). On continue.")
        continue

print(f"\nTotal livres récupérés : {len(all_books)}")


Pages totales : 50
✅ Page 1: 20 livres
✅ Page 2: 20 livres
✅ Page 3: 20 livres
✅ Page 4: 20 livres
✅ Page 5: 20 livres
✅ Page 6: 20 livres
✅ Page 7: 20 livres
✅ Page 8: 20 livres
✅ Page 9: 20 livres
✅ Page 10: 20 livres
✅ Page 11: 20 livres
✅ Page 12: 20 livres
✅ Page 13: 20 livres
✅ Page 14: 20 livres
✅ Page 15: 20 livres
✅ Page 16: 20 livres
✅ Page 17: 20 livres
✅ Page 18: 20 livres
✅ Page 19: 20 livres
✅ Page 20: 20 livres
✅ Page 21: 20 livres
✅ Page 22: 20 livres
✅ Page 23: 20 livres
✅ Page 24: 20 livres
✅ Page 25: 20 livres
✅ Page 26: 20 livres
✅ Page 27: 20 livres
✅ Page 28: 20 livres
✅ Page 29: 20 livres
✅ Page 30: 20 livres
✅ Page 31: 20 livres
✅ Page 32: 20 livres
✅ Page 33: 20 livres
✅ Page 34: 20 livres
✅ Page 35: 20 livres
✅ Page 36: 20 livres
✅ Page 37: 20 livres
✅ Page 38: 20 livres
✅ Page 39: 20 livres
✅ Page 40: 20 livres
✅ Page 41: 20 livres
✅ Page 42: 20 livres
✅ Page 43: 20 livres
✅ Page 44: 20 livres
✅ Page 45: 20 livres
✅ Page 46: 20 livres
✅ Page 47: 20 livres
✅ P

In [24]:
# scrape_books_page_of_y.py
import re
import time
import csv
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

BASE = "https://books.toscrape.com/catalogue/"
FIRST_URL = urljoin(BASE, "page-1.html")
HEADERS = {"User-Agent": "Mozilla/5.0"}

def fetch_soup(url: str) -> BeautifulSoup:
    r = requests.get(url, timeout=15, headers=HEADERS)
    r.raise_for_status()
    return BeautifulSoup(r.text, "html5lib")

def get_total_pages() -> int:
    """Lit 'Page 1 of Y' dans <li class="current"> pour connaître Y."""
    soup = fetch_soup(FIRST_URL)
    li = soup.select_one("li.current")
    if not li:
        # fallback: si pas d’info, on considère qu’il n’y a qu’une page
        return 1
    text = li.get_text(strip=True)  # ex: 'Page 1 of 50'
    m = re.search(r"Page\s+\d+\s+of\s+(\d+)", text, re.I)
    return int(m.group(1)) if m else 1

def parse_page(n: int) -> list[dict]:
    """Récupère (title, price_text, price) pour la page n."""
    url = urljoin(BASE, f"page-{n}.html")
    soup = fetch_soup(url)

    rows = []
    for art in soup.select("article.product_pod"):
        # Titre
        a = art.select_one("h3 a")
        title = (a.get("title") or a.get_text(strip=True)) if a else None

        # Prix (texte)
        price_tag = art.select_one("p.price_color")
        price_text = price_tag.get_text(strip=True) if price_tag else None

        # Prix (numérique) — enlève symbole £ et convertit
        price_num = None
        if price_text:
            m = re.search(r"([\d]+\.\d+)", price_text.replace(",", "."))
            if m:
                price_num = float(m.group(1))

        rows.append({"title": title, "price_text": price_text, "price": price_num})
    return rows

def main():
    total_pages = get_total_pages()
    print(f"📄 Pages totales détectées : {total_pages}")

    all_books = []
    for i in range(1, total_pages + 1):
        try:
            page_rows = parse_page(i)
            all_books.extend(page_rows)
            print(f"✅ Page {i}/{total_pages} : {len(page_rows)} livres")
        except requests.RequestException as e:
            print(f"⚠️ Page {i} ignorée (erreur réseau: {e})")
        time.sleep(0.25)  # politesse réseau

    print(f"\n📚 Total livres collectés : {len(all_books)}")

    # Export CSV
    out_path = "books_all_titles_prices.csv"
    with open(out_path, "w", encoding="utf-8", newline="") as f:
        w = csv.DictWriter(f, fieldnames=["title", "price_text", "price"])
        w.writeheader()
        w.writerows(all_books)

    # Petit aperçu
    for row in all_books[:5]:
        print(row)

    print(f"\n💾 Export terminé → {out_path}")

if __name__ == "__main__":
    main()


📄 Pages totales détectées : 50
✅ Page 1/50 : 20 livres
✅ Page 2/50 : 20 livres
✅ Page 3/50 : 20 livres
✅ Page 4/50 : 20 livres
✅ Page 5/50 : 20 livres
✅ Page 6/50 : 20 livres
✅ Page 7/50 : 20 livres
✅ Page 8/50 : 20 livres
✅ Page 9/50 : 20 livres
✅ Page 10/50 : 20 livres
✅ Page 11/50 : 20 livres
✅ Page 12/50 : 20 livres
✅ Page 13/50 : 20 livres
✅ Page 14/50 : 20 livres
✅ Page 15/50 : 20 livres
✅ Page 16/50 : 20 livres
✅ Page 17/50 : 20 livres
✅ Page 18/50 : 20 livres
✅ Page 19/50 : 20 livres
✅ Page 20/50 : 20 livres
✅ Page 21/50 : 20 livres
✅ Page 22/50 : 20 livres
✅ Page 23/50 : 20 livres
✅ Page 24/50 : 20 livres
✅ Page 25/50 : 20 livres
✅ Page 26/50 : 20 livres
✅ Page 27/50 : 20 livres
✅ Page 28/50 : 20 livres
✅ Page 29/50 : 20 livres
✅ Page 30/50 : 20 livres
✅ Page 31/50 : 20 livres
✅ Page 32/50 : 20 livres
✅ Page 33/50 : 20 livres
✅ Page 34/50 : 20 livres
✅ Page 35/50 : 20 livres
✅ Page 36/50 : 20 livres
✅ Page 37/50 : 20 livres
✅ Page 38/50 : 20 livres
✅ Page 39/50 : 20 livres
✅ P

In [25]:
# scrape_drapeaux_wikipedia.py
import os
import re
import time
import unicodedata
from concurrent.futures import ThreadPoolExecutor, as_completed
from urllib.parse import urljoin

import requests
from bs4 import BeautifulSoup

WIKI_URL = "https://fr.wikipedia.org/wiki/Galerie_des_drapeaux_des_pays_du_monde"
OUT_DIR = "drapeaux"
HEADERS = {"User-Agent": "Mozilla/5.0"}

# ---------- Utils ----------
def slugify(name: str) -> str:
    """Transforme 'Côte d’Ivoire' -> 'Cote_d_Ivoire' (nom de fichier sûr)."""
    name = unicodedata.normalize("NFKD", name).encode("ascii", "ignore").decode("ascii")
    name = re.sub(r"[^\w\s-]+", "", name)              # retire ponctuation
    name = re.sub(r"[\s-]+", "_", name.strip())        # espaces/traits -> _
    return name or "inconnu"

def wiki_thumb_to_original(src: str) -> str:
    """
    Convertit une miniature Wikipédia en URL de l’original.
    Ex:
      .../wikipedia/commons/thumb/a/ab/Flag_of_France.svg/1200px-Flag_of_France.svg.png
    ->   .../wikipedia/commons/a/ab/Flag_of_France.svg
    """
    if not src:
        return src
    # Ajouter https: si l’URL commence par //
    if src.startswith("//"):
        src = "https:" + src
    if "/thumb/" in src:
        prefix, rest = src.split("/thumb/", 1)  # '.../commons', 'a/ab/Flag_of_France.svg/1200px-...'
        parts = rest.split("/")
        # enlever le dernier segment (taille) pour retrouver le nom de fichier original
        original_path = "/".join(parts[:-1])    # 'a/ab/Flag_of_France.svg'
        return f"{prefix}/{original_path}"
    return src

def get_ext_from_url(u: str) -> str:
    m = re.search(r"\.([a-zA-Z0-9]+)(?:\?|#|$)", u)
    if m:
        ext = m.group(1).lower()
        # certaines SVG ont '?download' etc.
        if ext in {"svg", "png", "jpg", "jpeg", "gif", "webp"}:
            return "." + ext
    return ".img"  # fallback

def ensure_unique_path(base_path: str) -> str:
    """Ajoute un suffixe numérique si le fichier existe déjà."""
    if not os.path.exists(base_path):
        return base_path
    root, ext = os.path.splitext(base_path)
    i = 2
    while True:
        cand = f"{root}_{i}{ext}"
        if not os.path.exists(cand):
            return cand
        i += 1

# ---------- Scrape ----------
def fetch_soup(url: str) -> BeautifulSoup:
    r = requests.get(url, headers=HEADERS, timeout=20)
    r.raise_for_status()
    # html5lib est plus tolérant ; si non installé, remplace par "html.parser"
    return BeautifulSoup(r.text, "html5lib")

def parse_gallery_entries(soup: BeautifulSoup):
    """
    Retourne une liste de tuples (country_name, image_url_original)
    en lisant les blocs de galerie.
    """
    results = []
    # Les galeries Wikipédia utilisent souvent .gallery .gallerybox
    for box in soup.select(".gallery .gallerybox"):
        # Nom du pays/caption dans .gallerytext (souvent <a> ou texte)
        caption = box.select_one(".gallerytext")
        if not caption:
            continue
        # Récupérer un nom propre (priorité au texte du lien, sinon texte brut)
        link = caption.find("a")
        country_name = (link.get_text(" ", strip=True) if link else caption.get_text(" ", strip=True))
        country_name = re.sub(r"\s*\(.*?\)\s*$", "", country_name).strip()  # retire parenthèses finales

        # L'image miniature est dans .thumb img
        img = box.select_one(".thumb img")
        if not img:
            # parfois structure <div class="thumb"> absente ; fallback
            img = box.find("img")
        if not img or not img.get("src"):
            continue

        thumb_src = img["src"]
        orig_src = wiki_thumb_to_original(thumb_src)

        results.append((country_name, orig_src))
    return results

def download_one(name: str, url: str, out_dir: str) -> tuple[str, str, bool, str]:
    """
    Télécharge une image. Retourne (name, path, ok, msg).
    """
    try:
        ext = get_ext_from_url(url)
        fname = slugify(name) + ext
        os.makedirs(out_dir, exist_ok=True)
        out_path = ensure_unique_path(os.path.join(out_dir, fname))

        with requests.get(url, headers=HEADERS, timeout=30, stream=True) as r:
            r.raise_for_status()
            with open(out_path, "wb") as f:
                for chunk in r.iter_content(chunk_size=8192):
                    if chunk:
                        f.write(chunk)
        return (name, out_path, True, "ok")
    except Exception as e:
        return (name, url, False, str(e))

def main():
    print("➡️  Récupération de la page Wikipédia…")
    soup = fetch_soup(WIKI_URL)

    print("➡️  Analyse de la galerie…")
    entries = parse_gallery_entries(soup)
    # Déduplication simple par nom (si la page contient plusieurs variantes)
    seen = set()
    unique_entries = []
    for name, url in entries:
        key = slugify(name)
        if key not in seen and url:
            seen.add(key)
            unique_entries.append((name, url))

    print(f"🧾 Drapeaux détectés : {len(unique_entries)}")
    if not unique_entries:
        print("⚠️  Aucun drapeau détecté. La structure de la page a peut-être changé.")
        return

    print(f"⬇️  Téléchargement dans ./{OUT_DIR} …")
    ok_count = 0
    fail_count = 0
    # Télécharge en parallèle pour aller plus vite (reste raisonnable)
    with ThreadPoolExecutor(max_workers=8) as ex:
        futures = [ex.submit(download_one, name, url, OUT_DIR) for name, url in unique_entries]
        for fut in as_completed(futures):
            name, path_or_url, ok, msg = fut.result()
            if ok:
                ok_count += 1
                print(f"✅ {name} -> {path_or_url}")
            else:
                fail_count += 1
                print(f"❌ {name} ({path_or_url}) : {msg}")
            # petite pause douce au serveur (facultatif ici car stream + pool)
            time.sleep(0.02)

    print(f"\n📦 Terminé. Succès: {ok_count} | Échecs: {fail_count} | Dossier: {OUT_DIR}")

if __name__ == "__main__":
    main()


➡️  Récupération de la page Wikipédia…
➡️  Analyse de la galerie…
🧾 Drapeaux détectés : 0
⚠️  Aucun drapeau détecté. La structure de la page a peut-être changé.


In [39]:
import requests

url = "https://upload.wikimedia.org/wikipedia/commons/c/c3/Flag_of_France.svg"
r = requests.get(url, timeout=20)
r.raise_for_status()

with open("France.svg", "wb") as f:
    f.write(r.content)

print("✅ Drapeau de la France enregistré sous France.svg")


HTTPError: 403 Client Error: Forbidden for url: https://upload.wikimedia.org/wikipedia/commons/c/c3/Flag_of_France.svg

In [40]:
import requests

url = "https://upload.wikimedia.org/wikipedia/commons/c/c3/Flag_of_France.svg"

headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) "
                  "AppleWebKit/537.36 (KHTML, like Gecko) "
                  "Chrome/118.0.0.0 Safari/537.36"
}

r = requests.get(url, headers=headers, timeout=20)
r.raise_for_status()

with open("France.svg", "wb") as f:
    f.write(r.content)

print("✅ Drapeau de la France enregistré sous France.svg")


✅ Drapeau de la France enregistré sous France.svg


In [31]:
# scrape_all_flags.py
import os
import re
import time
import unicodedata
from urllib.parse import urljoin

import requests
from bs4 import BeautifulSoup

WIKI_URL = "https://fr.wikipedia.org/wiki/Galerie_des_drapeaux_des_pays_du_monde"
OUT_DIR = "drapeaux"

HEADERS = {
    # User-Agent "navigateur" pour éviter les 403 (même méthode que précédemment)
    "User-Agent": (
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/118.0.0.0 Safari/537.36"
    )
}

# ---------- Utilitaires ----------

def slugify(name: str) -> str:
    """Transforme 'Côte d’Ivoire' -> 'Cote_d_Ivoire' (nom de fichier sûr)."""
    name = unicodedata.normalize("NFKD", name).encode("ascii", "ignore").decode("ascii")
    name = re.sub(r"[^\w\s-]+", "", name)           # retire ponctuation
    name = re.sub(r"[\s-]+", "_", name.strip())     # espaces/traits -> _
    return name or "inconnu"

def wiki_thumb_to_original(src: str) -> str:
    """
    Convertit une miniature Wikipédia en URL de l’original.
    .../commons/thumb/a/ab/Flag_of_France.svg/1200px-Flag_of_France.svg.png
    ->  .../commons/a/ab/Flag_of_France.svg
    """
    if not src:
        return src
    if src.startswith("//"):
        src = "https:" + src
    if "/thumb/" in src:
        prefix, rest = src.split("/thumb/", 1)   # '.../commons', 'a/ab/Flag_of_.../1200px-...'
        parts = rest.split("/")
        original_path = "/".join(parts[:-1])     # retire le segment taille
        return f"{prefix}/{original_path}"
    return src

def get_ext_from_url(u: str) -> str:
    m = re.search(r"\.([a-zA-Z0-9]+)(?:\?|#|$)", u or "")
    if not m:
        return ".img"
    ext = m.group(1).lower()
    return "." + ext if ext in {"svg", "png", "jpg", "jpeg", "gif", "webp"} else ".img"

# ---------- Scrape ----------

def fetch_soup(url: str) -> BeautifulSoup:
    r = requests.get(url, headers=HEADERS, timeout=30)
    r.raise_for_status()
    # Si html5lib n'est pas installé, remplace "html5lib" par "html.parser"
    return BeautifulSoup(r.text, "html5lib")

def parse_gallery_entries(soup: BeautifulSoup):
    """
    Retourne [(country_name, image_url_original)] depuis les blocs .gallerybox.
    """
    entries = []
    for box in soup.select(".gallery .gallerybox"):
        # Nom du pays : priorité au lien dans .gallerytext
        caption = box.select_one(".gallerytext")
        if not caption:
            continue
        a = caption.find("a")
        name = (a.get_text(" ", strip=True) if a else caption.get_text(" ", strip=True))
        # Nettoie les parenthèses de fin éventuelles (ex: "États-Unis (USA)")
        name = re.sub(r"\s*\(.*?\)\s*$", "", name).strip()

        # Image miniature dans .thumb img
        img = box.select_one(".thumb img") or box.find("img")
        if not img or not img.get("src"):
            continue
        orig_url = wiki_thumb_to_original(img["src"])
        entries.append((name, orig_url))
    return entries

def download(url: str, out_path: str, retries: int = 2, backoff: float = 0.8) -> None:
    last_exc = None
    for attempt in range(retries + 1):
        try:
            with requests.get(url, headers=HEADERS, timeout=60, stream=True) as r:
                r.raise_for_status()
                with open(out_path, "wb") as f:
                    for chunk in r.iter_content(8192):
                        if chunk:
                            f.write(chunk)
            return
        except Exception as e:
            last_exc = e
            time.sleep(backoff * (attempt + 1))
    raise last_exc

def main():
    os.makedirs(OUT_DIR, exist_ok=True)
    print("➡️  Récupération de la page Wikipédia…")
    soup = fetch_soup(WIKI_URL)

    print("➡️  Extraction de la galerie…")
    entries = parse_gallery_entries(soup)
    if not entries:
        print("⚠️  Aucun drapeau détecté (la structure a peut-être changé).")
        return

    # Déduplique par nom de pays
    seen = set()
    uniq = []
    for name, url in entries:
        key = slugify(name)
        if key not in seen:
            seen.add(key)
            uniq.append((name, url))

    print(f"🧾 Drapeaux détectés : {len(uniq)}")
    ok, ko = 0, 0

    for name, url in uniq:
        ext = get_ext_from_url(url)
        fname = f"{slugify(name)}{ext}"
        out_path = os.path.join(OUT_DIR, fname)
        try:
            download(url, out_path)
            ok += 1
            print(f"✅ {name} -> {out_path}")
        except Exception as e:
            ko += 1
            print(f"❌ {name} ({url}) : {e}")
        time.sleep(0.05)  # petite politesse réseau

    print(f"\n📦 Terminé. Succès: {ok} | Échecs: {ko} | Dossier: {OUT_DIR}")

if __name__ == "__main__":
    main()


➡️  Récupération de la page Wikipédia…
➡️  Extraction de la galerie…
⚠️  Aucun drapeau détecté (la structure a peut-être changé).


In [33]:
import re
import requests
from bs4 import BeautifulSoup

URL = "https://fr.wikipedia.org/wiki/Galerie_des_drapeaux_des_pays_du_monde"
HEADERS = {
    "User-Agent": (
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/118.0.0.0 Safari/537.36"
    )
}

def thumb_to_original(src: str) -> str:
    """Convertit une miniature Wikimedia en URL de l’original."""
    if not src:
        return src
    if src.startswith("//"):
        src = "https:" + src
    if "/thumb/" in src:
        prefix, rest = src.split("/thumb/", 1)   # …/commons , a/ab/Flag_of_…/1200px-…
        parts = rest.split("/")
        original_path = "/".join(parts[:-1])     # retire le segment '1200px-...'
        return f"{prefix}/{original_path}"
    return src

# 1) Télécharger la page avec User-Agent
resp = requests.get(URL, headers=HEADERS, timeout=30)
resp.raise_for_status()
soup = BeautifulSoup(resp.text, "html5lib")  # ou "html.parser" si html5lib non installé

# 2) Repérer la séquence de liens: [Image] [Drapeau] [France]
content = soup.select_one("#mw-content-text")
links = content.select("a") if content else []

img_href = None
for i, a in enumerate(links):
    if a.get_text(strip=True) == "Drapeau":
        # lien suivant = pays
        if i + 1 < len(links) and re.search(r"\bFrance\b", links[i+1].get_text(" ", strip=True), re.I):
            # lien précédent = "Image" -> miniature
            if i - 1 >= 0 and links[i-1].get_text(strip=True) == "Image":
                img_href = links[i-1].get("href")
                break

if not img_href:
    raise RuntimeError("Impossible de trouver la vignette du drapeau de la France sur la page.")

# 3) Convertir miniature -> original (SVG/PNG)
img_url = thumb_to_original(img_href)

# 4) Télécharger l’image finale avec le même User-Agent
r = requests.get(img_url, headers=HEADERS, timeout=30)
r.raise_for_status()
ext = ".svg" if ".svg" in img_url.lower() else ".png"
with open(f"France{ext}", "wb") as f:
    f.write(r.content)

print(f"✅ Drapeau de la France enregistré sous France{ext}")


RuntimeError: Impossible de trouver la vignette du drapeau de la France sur la page.

In [35]:
import re
import requests
from bs4 import BeautifulSoup

URL = "https://fr.wikipedia.org/wiki/Galerie_des_drapeaux_des_pays_du_monde"

HEADERS = {
    "User-Agent": (
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/118.0.0.0 Safari/537.36"
    )
}

def thumb_to_original(src: str) -> str:
    """Convertit une miniature Wikimedia en URL d’image originale."""
    if not src:
        return src
    if src.startswith("//"):
        src = "https:" + src
    if "/thumb/" in src:
        prefix, rest = src.split("/thumb/", 1)
        parts = rest.split("/")
        original_path = "/".join(parts[:-1])
        return f"{prefix}/{original_path}"
    return src

# 1️⃣ Télécharger la page Wikipédia (avec User-Agent)
resp = requests.get(URL, headers=HEADERS, timeout=30)
resp.raise_for_status()
soup = BeautifulSoup(resp.text, "html5lib")  # ou "html.parser"

# 2️⃣ Récupérer tous les liens dans le contenu principal
content = soup.select_one("#mw-content-text")
links = content.select("a") if content else []

# 3️⃣ Identifier les triplets [Image] [Drapeau] [Nom du pays]
flags = []
for i, a in enumerate(links):
    if a.get_text(strip=True) == "Drapeau":
        if (i - 1 >= 0 and links[i - 1].get_text(strip=True) == "Image") and (i + 1 < len(links)):
            country = links[i + 1].get_text(" ", strip=True)
            img_href = links[i - 1].get("href")
            if img_href and country:
                full_url = thumb_to_original(img_href)
                flags.append((country, full_url))

# 4️⃣ Supprimer les doublons
unique_flags = []
seen = set()
for country, url in flags:
    if country not in seen:
        seen.add(country)
        unique_flags.append((country, url))

# 5️⃣ Afficher la liste complète
print(f"🌍 {len(unique_flags)} drapeaux trouvés :\n")
for country, url in unique_flags:
    print(f"- {country}: {url}")


🌍 0 drapeaux trouvés :



In [36]:
import re
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

BASE = "https://fr.wikipedia.org"
URL  = f"{BASE}/wiki/Galerie_des_drapeaux_des_pays_du_monde"

HEADERS = {
    "User-Agent": (
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/118.0.0.0 Safari/537.36"
    ),
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
}

def fetch_soup(url: str) -> BeautifulSoup:
    r = requests.get(url, headers=HEADERS, timeout=30)
    r.raise_for_status()
    return BeautifulSoup(r.text, "html.parser")  # ou "html5lib" si installé

def resolve_original_from_filepage(filepage_href: str) -> str | None:
    """Depuis une page fichier /wiki/Fichier:..., récupère le lien du média original."""
    filepage_url = filepage_href if filepage_href.startswith("http") else urljoin(BASE, filepage_href)
    s = fetch_soup(filepage_url)
    a = s.select_one("div.fullMedia a")  # lien direct vers l'original
    if not a:
        return None
    href = a.get("href")
    if not href:
        return None
    # Sur Commons, souvent //upload.wikimedia.org/...
    if href.startswith("//"):
        href = "https:" + href
    elif href.startswith("/"):
        href = urljoin(BASE, href)
    return href

# 1) Charger la page galerie
soup = fetch_soup(URL)

# 2) Parcourir le contenu principal et extraire les triplets Image / Drapeau / Pays
content = soup.select_one("#mw-content-text")
links = content.select("a") if content else []

pairs = []
for i, a in enumerate(links):
    if a.get_text(strip=True) == "Drapeau":
        # lien précédent "Image" -> page fichier
        if i - 1 >= 0 and links[i-1].get_text(strip=True) == "Image":
            filepage_href = links[i-1].get("href")
            # lien suivant -> nom du pays
            if i + 1 < len(links):
                country = links[i+1].get_text(" ", strip=True)
                if filepage_href and country:
                    pairs.append((country, filepage_href))

# 3) Dédupliquer par pays
seen = set()
unique_pairs = []
for country, filepage_href in pairs:
    if country not in seen:
        seen.add(country)
        unique_pairs.append((country, filepage_href))

# 4) Résoudre chaque page fichier en URL d’image originale
results = []
for country, filepage_href in unique_pairs:
    original = resolve_original_from_filepage(filepage_href)
    if original:
        results.append((country, original))

# 5) Affichage
print(f"🌍 {len(results)} drapeaux résolus :\n")
for country, url in results:
    print(f"- {country}: {url}")


🌍 0 drapeaux résolus :



In [37]:
import re
import time
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

BASE = "https://fr.wikipedia.org"
URL  = f"{BASE}/wiki/Galerie_des_drapeaux_des_pays_du_monde"

HEADERS = {
    "User-Agent": (
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/118.0.0.0 Safari/537.36"
    )
}

def fetch_soup(url: str) -> BeautifulSoup:
    r = requests.get(url, headers=HEADERS, timeout=30)
    r.raise_for_status()
    # html5lib si installé; sinon "html.parser"
    return BeautifulSoup(r.text, "html.parser")

def resolve_original_from_filepage(filepage_href: str) -> str | None:
    """Depuis /wiki/Fichier:..., récupérer l’URL du média original."""
    filepage_url = filepage_href if filepage_href.startswith("http") else urljoin(BASE, filepage_href)
    s = fetch_soup(filepage_url)
    a = s.select_one("div.fullMedia a")
    if not a:
        return None
    href = a.get("href")
    if not href:
        return None
    if href.startswith("//"):
        href = "https:" + href
    elif href.startswith("/"):
        href = urljoin(BASE, href)
    return href

def extract_country_from_li(li: BeautifulSoup) -> str | None:
    """
    Priorités pour trouver le nom du pays dans un <li> :
    1) le lien qui suit immédiatement le lien dont le texte est 'Drapeau'
    2) sinon le dernier lien non-image du <li>
    """
    links = li.select("a")
    # cas 1: 'Drapeau' puis le lien suivant = pays
    for i, a in enumerate(links):
        if a.get_text(strip=True) == "Drapeau" and i + 1 < len(links):
            txt = links[i + 1].get_text(" ", strip=True)
            if txt and txt.lower() != "image":
                return txt

    # cas 2: fallback = dernier lien non-image (souvent le pays)
    candidates = [a for a in links if "image" not in (a.get("class") or [])]
    if candidates:
        return candidates[-1].get_text(" ", strip=True)

    return None

def main():
    soup = fetch_soup(URL)
    content = soup.select_one("#mw-content-text .mw-parser-output")
    if not content:
        print("⚠️ Zone de contenu introuvable.")
        return

    results = []
    items = content.select("li")
    for li in items:
        img_link = li.select_one("a.image")  # lien vers /wiki/Fichier:...
        if not img_link or not img_link.get("href"):
            continue

        country = extract_country_from_li(li)
        if not country:
            continue

        filepage_href = img_link.get("href")
        original = resolve_original_from_filepage(filepage_href)
        if original:
            results.append((country, original))
        time.sleep(0.05)  # douceur serveur

    # dédoublonnage basique par nom
    unique = []
    seen = set()
    for country, url in results:
        key = country.strip()
        if key not in seen:
            seen.add(key)
            unique.append((country, url))

    print(f"🌍 {len(unique)} drapeaux résolus :\n")
    for country, url in unique:
        print(f"- {country}: {url}")

if __name__ == "__main__":
    main()


🌍 0 drapeaux résolus :



In [38]:
import requests

country = "France"
api = (
    "https://commons.wikimedia.org/w/api.php?"
    "action=query&titles=File:Flag_of_{0}.svg&prop=imageinfo&"
    "iiprop=url&format=json"
).format(country.replace(" ", "_"))

r = requests.get(api, headers={"User-Agent": "MonScript/1.0"})
data = r.json()
pages = data["query"]["pages"]
for _, p in pages.items():
    print(p["imageinfo"][0]["url"])


https://upload.wikimedia.org/wikipedia/commons/c/c3/Flag_of_France.svg


In [41]:
import requests
from bs4 import BeautifulSoup
import os

url = "https://en.wikipedia.org/wiki/List_of_national_flags_of_sovereign_states"
response = requests.get(url)
soup = BeautifulSoup(response.content, "html.parser")

# Crée un dossier pour les drapeaux
os.makedirs("flags_hd", exist_ok=True)

# Trouve toutes les images de drapeaux
for img in soup.find_all("img"):
    src = img.get("src")
    if "Flag_of" in src and src.startswith("//upload.wikimedia.org"):
        full_url = "https:" + src.replace("/thumb", "").rsplit("/", 1)[0]
        filename = full_url.split("/")[-1]
        img_data = requests.get(full_url).content
        with open(f"flags_hd/{filename}", "wb") as f:
            f.write(img_data)


/var/folders/zf/zpmmyk0s1mdgk_k18jbpmycc0000gn/T/ipykernel_26601/535741495.py:7: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(response.content, "html.parser")


In [43]:
import os
import requests
from bs4 import BeautifulSoup

# URL de la page Wikipedia
URL = "https://en.wikipedia.org/wiki/List_of_national_flags_of_sovereign_states"

# Crée le dossier 'drapeaux' s'il n'existe pas
os.makedirs("drapeaux", exist_ok=True)

# Récupère le contenu HTML
response = requests.get(URL)
soup = BeautifulSoup(response.content, "html.parser")

# Recherche toutes les images de drapeaux
flags = soup.select("table.wikitable img")

for img in flags:
    src = img.get("src")
    if src and "Flag_of" in src:
        # Construit l'URL vers l'image HD
        hd_url = "https:" + src.split("/thumb")[0] + "/" + src.split("/")[-1]
        filename = hd_url.split("/")[-1]

        # Télécharge l'image
        try:
            img_data = requests.get(hd_url).content
            with open(f"drapeaux/{filename}", "wb") as f:
                f.write(img_data)
            print(f"Téléchargé : {filename}")
        except Exception as e:
            print(f"Erreur pour {filename} : {e}")


/var/folders/zf/zpmmyk0s1mdgk_k18jbpmycc0000gn/T/ipykernel_26601/3319640986.py:13: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(response.content, "html.parser")


In [45]:
import requests
import os

# Crée le dossier 'drapeaux' s'il n'existe pas
os.makedirs("drapeaux", exist_ok=True)

# Liste des URLs HD des drapeaux
drapeaux = {
    "France": "https://upload.wikimedia.org/wikipedia/commons/c/c3/Flag_of_France.svg",
    "Germany": "https://upload.wikimedia.org/wikipedia/commons/b/ba/Flag_of_Germany.svg"
}

# Téléchargement
for pays, url in drapeaux.items():
    filename = f"drapeaux/{pays.replace(' ', '_')}.svg"
    try:
        response = requests.get(url)
        with open(filename, "wb") as f:
            f.write(response.content)
        print(f"Téléchargé : {filename}")
    except Exception as e:
        print(f"Erreur pour {pays} : {e}")


Téléchargé : drapeaux/France.svg
Téléchargé : drapeaux/Germany.svg


In [48]:
import os
import requests
from bs4 import BeautifulSoup

# URL de la page Wikipedia
URL = "https://en.wikipedia.org/wiki/List_of_national_flags_of_sovereign_states"

# Crée le dossier 'drapeaux' s'il n'existe pas
os.makedirs("drapeaux", exist_ok=True)

# Récupère le contenu HTML
response = requests.get(URL)
soup = BeautifulSoup(response.content, "html.parser")

# Sélectionne les images de drapeaux dans les tableaux
images = soup.select("table.wikitable img")

# Limite à 3 drapeaux
for i, img in enumerate(images[:3]):
    src = img.get("src")
    if src and "Flag_of" in src:
        # Construit l'URL vers le fichier SVG original
        parts = src.split("/thumb")
        if len(parts) == 2:
            hd_url = "https:" + parts[0] + "/" + parts[1].split("/")[-1]
            filename = hd_url.split("/")[-1]

            # Télécharge l'image
            try:
                img_data = requests.get(hd_url).content
                with open(f"drapeaux/{filename}", "wb") as f:
                    f.write(img_data)
                print(f"Téléchargé : {filename}")
            except Exception as e:
                print(f"Erreur pour {filename} : {e}")


/var/folders/zf/zpmmyk0s1mdgk_k18jbpmycc0000gn/T/ipykernel_26601/1946834342.py:13: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(response.content, "html.parser")


In [50]:
npm install puppeteer

SyntaxError: invalid syntax (3127857321.py, line 1)

In [51]:
!pip install playwright
!playwright install

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.7/38.7 MB 10.6 MB/s eta 0:00:0000:0100:01
129.7 MiB [                    ] 0% 0.0s129.7 MiB [                    ] 0% 282.3s129.7 MiB [                    ] 0% 312.1s129.7 MiB [                    ] 0% 297.2s129.7 MiB [                    ] 0% 349.1s129.7 MiB [                    ] 0% 200.8s129.7 MiB [                    ] 0% 237.5s129.7 MiB [                    ] 0% 188.8s129.7 MiB [                    ] 0% 121.6s129.7 MiB [                    ] 0% 116.5s129.7 MiB [                    ] 0% 104.8s129.7 MiB [                    ] 0% 106.6s129.7 MiB [                    ] 0% 106.1s129.7 MiB [                    ] 1% 77.5s129.7 MiB [                    ] 1% 76.5s129.7 MiB [                    ] 1% 73.9s129.7 MiB [                    ] 1% 74.1s129.7 MiB [                    ] 1% 72.5s129.7 MiB [                    ] 1% 73.8s129.7 MiB [                    ] 1% 67.8s129.7 MiB [                    ] 1% 67.6s129.7 MiB [                    ] 1% 66.

In [52]:
import asyncio
from playwright.async_api import async_playwright

async def get_first_quote():
    """
    Lance un navigateur sans tête, navigue vers la page JS-loaded, 
    attend que le contenu apparaisse, et retourne la première citation.
    """
    url = "https://quotes.toscrape.com/js/"
    quote_selector = ".quote .text"  # Le sélecteur CSS de la citation

    print("Démarrage du navigateur...")
    async with async_playwright() as p:
        # Lancer le navigateur Chromium en mode "headless" (sans interface graphique visible)
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()

        print(f"Navigation vers : {url}")
        # Aller à l'URL. Ceci déclenche l'exécution du JavaScript.
        await page.goto(url)

        # ❗ Étape Cruciale : Attendre que l'élément de citation soit chargé par le JS.
        print("Attente du chargement du contenu dynamique...")
        try:
            await page.wait_for_selector(quote_selector, timeout=10000)  # Timeout de 10 secondes
        except Exception as e:
            await browser.close()
            print("Erreur : Le sélecteur de citation n'est pas apparu à temps.")
            return "Erreur d'extraction (Contenu non chargé)"

        # Extraire le texte de la première citation correspondant au sélecteur
        first_quote_element = await page.locator(quote_selector).first.inner_text()
        
        # Fermer le navigateur
        await browser.close()
        print("Navigateur fermé.")
        
        return first_quote_element

# Exécuter la fonction asynchrone et afficher le résultat
# La fonction asyncio.run() est nécessaire pour lancer le code asynchrone dans un environnement synchrone (comme un Notebook)
first_quote = await get_first_quote() 

print("\n-----------------------------------------")
print("✅ Première citation récupérée :")
print(f'"{first_quote}"')
print("-----------------------------------------")

Démarrage du navigateur...
Navigation vers : https://quotes.toscrape.com/js/
Attente du chargement du contenu dynamique...
Navigateur fermé.

-----------------------------------------
✅ Première citation récupérée :
"“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”"
-----------------------------------------


In [65]:
import requests
from bs4 import BeautifulSoup
import os
import re
import time 

# Constantes 
URL_WIKIPEDIA = "https://fr.wikipedia.org/wiki/Galerie_des_drapeaux_des_pays_du_monde"
BASE_URL = "https://fr.wikipedia.org"

# User-Agent robuste pour simuler un navigateur
HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

DOSSIER_CIBLE = "drapeaux"

def telecharger_drapeaux_svg_debug_final(url_principale, dossier_cible):
    """
    Scrape avec prints à chaque étape critique pour le diagnostic.
    """
    
    if not os.path.exists(dossier_cible):
        os.makedirs(dossier_cible)
        print(f"[SETUP] Dossier '{dossier_cible}' créé.")
    
    # --- ÉTAPE 1 : Récupérer les liens des pages de description des fichiers ---
    print("\n--- DÉBOGAGE ÉTAPE 1 : ACCÈS À LA PAGE PRINCIPALE ---")
    try:
        print(f"[DEBUG 1.0] Requête GET lancée vers: {url_principale}")
        response = requests.get(url_principale, headers=HEADERS)
        
        print(f"[DEBUG 1.1] Statut HTTP de la page principale: {response.status_code}")
        response.raise_for_status()
        print("[DEBUG 1.2] Connexion à la page principale réussie (Statut 2xx).")
    except requests.exceptions.RequestException as e:
        print(f"[ERREUR FATALE] Échec de l'accès à la page principale: {e}")
        return

    soup = BeautifulSoup(response.text, 'html.parser')
    
    # NOUVEAU SELECTEUR ROBUSTE: Cibler tous les liens de fichiers dans le contenu principal
    selector = '.mw-parser-output a[href^="/wiki/Fichier:"]'
    file_page_links = soup.select(selector)
    
    # Filtrer les liens qui ne sont pas des vignettes de drapeau (par exemple, liens dans les légendes)
    # On garde uniquement les liens qui ont un élément <img> enfant, assurant qu'ils sont visuels.
    valid_file_page_links = [link for link in file_page_links if link.find('img')]
    
    print(f"[DEBUG 1.3] Sélecteur CSS utilisé pour les liens de fichiers: '{selector}'")
    
    if not valid_file_page_links:
        print("-----------------------------------------------------------------")
        print("❌ Échec : Aucun lien de page de fichier de drapeau trouvé. Vérifiez le sélecteur CSS.")
        print("-----------------------------------------------------------------")
        return

    print(f"[DEBUG 1.4] {len(valid_file_page_links)} liens de pages de fichiers trouvés. Lancement du loop...")
    print("-----------------------------------------------------------------")
    
    drapeaux_telecharges = 0
    
    # --- LOOP PAR DRAPEAU ---
    for i, link in enumerate(valid_file_page_links):
        file_page_path = link['href']
        file_page_url = BASE_URL + file_page_path
        
        nom_fichier_brut = os.path.basename(file_page_path).replace('Fichier:', '')
        # Assurez-vous que le nom du fichier se termine par .svg pour le téléchargement
        nom_fichier = nom_fichier_brut if nom_fichier_brut.lower().endswith(('.svg', '.png', '.jpg')) else nom_fichier_brut + '.svg'
        
        print(f"\n--- DRAPEAU [{i+1}/{len(valid_file_page_links)}] : {nom_fichier} ---")
        
        try:
            # Étape 2 : Visiter la page de description du fichier
            print(f"[DEBUG 2.0] Requête GET lancée vers: {file_page_url}")
            file_response = requests.get(file_page_url, headers=HEADERS)
            
            print(f"[DEBUG 2.1] Statut HTTP de la page fichier: {file_response.status_code}")
            file_response.raise_for_status()
            print("[DEBUG 2.2] Connexion à la page fichier réussie (Statut 2xx).")
            
            file_soup = BeautifulSoup(file_response.text, 'html.parser')
            
            # --- ÉTAPE 3 : Trouver le lien du fichier original ---
            # Sélecteur du lien "Afficher dans le navigateur" que vous avez identifié
            selector_svg = 'a.cdx-docs-link[href*="upload.wikimedia.org"]'
            svg_link_element = file_soup.select_one(selector_svg)
            
            print(f"[DEBUG 3.0] Sélecteur pour lien SVG utilisé: '{selector_svg}'")
            
            if not svg_link_element:
                print("[ERREUR 3.1] ❌ Lien SVG non trouvé sur la page de description.")
                continue
            
            svg_url = svg_link_element['href']
            print(f"[DEBUG 3.2] Lien SVG/Upload trouvé: {svg_url}")
            
            chemin_fichier = os.path.join(dossier_cible, nom_fichier)
            
            # Étape 4 : Téléchargement du fichier
            print(f"[DEBUG 4.0] Requête GET lancée vers: {svg_url}")
            image_response = requests.get(svg_url, headers=HEADERS)
            
            print(f"[DEBUG 4.1] Statut HTTP du téléchargement: {image_response.status_code}")
            image_response.raise_for_status()
            
            with open(chemin_fichier, 'wb') as f:
                f.write(image_response.content)
            
            drapeaux_telecharges += 1
            print(f"[OK] Fichier téléchargé et enregistré : {chemin_fichier}")
            
            # Pause de 0.5s
            time.sleep(0.5) 
            
        except requests.exceptions.RequestException as e:
            print(f"[ERREUR] Échec du traitement ou du téléchargement de ce drapeau: {e}")
            time.sleep(1) 
            
    # --- FIN DU LOOP ---
    print("\n-----------------------------------------")
    print(f"✅ Terminé. **{drapeaux_telecharges}** fichiers SVG de haute résolution téléchargés.")
    print("-----------------------------------------")

# --- EXÉCUTION ---
telecharger_drapeaux_svg_debug_final(URL_WIKIPEDIA, DOSSIER_CIBLE)


--- DÉBOGAGE ÉTAPE 1 : ACCÈS À LA PAGE PRINCIPALE ---
[DEBUG 1.0] Requête GET lancée vers: https://fr.wikipedia.org/wiki/Galerie_des_drapeaux_des_pays_du_monde
[DEBUG 1.1] Statut HTTP de la page principale: 200
[DEBUG 1.2] Connexion à la page principale réussie (Statut 2xx).
[DEBUG 1.3] Sélecteur CSS utilisé pour les liens de fichiers: '.mw-parser-output a[href^="/wiki/Fichier:"]'
[DEBUG 1.4] 199 liens de pages de fichiers trouvés. Lancement du loop...
-----------------------------------------------------------------

--- DRAPEAU [1/199] : 2017-fr.wp-orange-source.svg ---
[DEBUG 2.0] Requête GET lancée vers: https://fr.wikipedia.org/wiki/Fichier:2017-fr.wp-orange-source.svg
[DEBUG 2.1] Statut HTTP de la page fichier: 200
[DEBUG 2.2] Connexion à la page fichier réussie (Statut 2xx).
[DEBUG 3.0] Sélecteur pour lien SVG utilisé: 'a.cdx-docs-link[href*="upload.wikimedia.org"]'
[ERREUR 3.1] ❌ Lien SVG non trouvé sur la page de description.

--- DRAPEAU [2/199] : Flags-1900.jpg ---
[DEBUG 2

In [66]:
import requests
from bs4 import BeautifulSoup
import os
import re
import time 

# Constantes 
URL_WIKIPEDIA = "https://fr.wikipedia.org/wiki/Galerie_des_drapeaux_des_pays_du_monde"
BASE_URL = "https://fr.wikipedia.org"

# User-Agent robuste pour simuler un navigateur
HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

DOSSIER_CIBLE = "drapeaux"

def telecharger_drapeaux_svg_final_v3(url_principale, dossier_cible):
    """
    Scrape avec prints à chaque étape critique pour le diagnostic.
    """
    
    if not os.path.exists(dossier_cible):
        os.makedirs(dossier_cible)
        print(f"[SETUP] Dossier '{dossier_cible}' créé.")
    
    # --- ÉTAPE 1 : Récupérer les liens des pages de description des fichiers ---
    print("\n--- DÉBOGAGE ÉTAPE 1 : ACCÈS À LA PAGE PRINCIPALE ---")
    try:
        response = requests.get(url_principale, headers=HEADERS)
        response.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(f"[ERREUR FATALE] Échec de l'accès à la page principale: {e}")
        return

    soup = BeautifulSoup(response.text, 'html.parser')
    
    # SÉLECTEUR ROBUSTE pour la page principale (Contenu + Fichier:)
    selector_main = '.mw-parser-output a[href^="/wiki/Fichier:"]'
    file_page_links = soup.select(selector_main)
    valid_file_page_links = [link for link in file_page_links if link.find('img')]
    
    if not valid_file_page_links:
        print("❌ Échec : Aucun lien de page de fichier de drapeau trouvé. Vérifiez le sélecteur CSS.")
        return

    print(f"[INFO] {len(valid_file_page_links)} liens de pages de fichiers trouvés. Lancement du loop...")
    print("-----------------------------------------------------------------")
    
    drapeaux_telecharges = 0
    
    # --- LOOP PAR DRAPEAU ---
    for i, link in enumerate(valid_file_page_links):
        file_page_path = link['href']
        file_page_url = BASE_URL + file_page_path
        
        nom_fichier_brut = os.path.basename(file_page_path).replace('Fichier:', '')
        nom_fichier = nom_fichier_brut if nom_fichier_brut.lower().endswith(('.svg', '.png', '.jpg')) else nom_fichier_brut + '.svg'
        
        # print(f"\n--- DRAPEAU [{i+1}/{len(valid_file_page_links)}] : {nom_fichier} ---") # Mettez en commentaire le print pour une exécution plus rapide
        
        try:
            # Étape 2 : Visiter la page de description du fichier
            file_response = requests.get(file_page_url, headers=HEADERS)
            file_response.raise_for_status()
            file_soup = BeautifulSoup(file_response.text, 'html.parser')
            
            # --- ÉTAPE 3 : Trouver le lien du fichier original (NOUVELLE CORRECTION) ---
            # 1. Tenter l'ancien sélecteur "Afficher dans le navigateur" au cas où
            selector_svg = 'a.cdx-docs-link[href*="upload.wikimedia.org"]'
            svg_link_element = file_soup.select_one(selector_svg)
            
            # 2. Si le lien cdx-docs-link est manquant, chercher le lien 'fullMedia' (lien historique vers le fichier source)
            if not svg_link_element:
                selector_svg_alt = '.fullMedia a[href*="upload.wikimedia.org"]'
                svg_link_element = file_soup.select_one(selector_svg_alt)
            
            if not svg_link_element:
                print(f"[Sauter] ❌ Lien SVG non trouvé pour {nom_fichier} (Ni cdx-docs-link, ni fullMedia).")
                continue
            
            svg_url = svg_link_element['href']
            
            # Nettoyer l'URL si elle contient le paramètre de téléchargement
            if '?download' in svg_url:
                svg_url = svg_url.split('?download')[0]
                
            # Vérifier que l'URL est complète si elle était relative (ne devrait pas être le cas ici, mais sécurité)
            if svg_url.startswith('//'):
                svg_url = 'https:' + svg_url
                
            # print(f"[DEBUG 3.2] Lien SVG/Upload trouvé: {svg_url}") # Décommenter si besoin
            
            chemin_fichier = os.path.join(dossier_cible, nom_fichier)
            
            # Étape 4 : Téléchargement du fichier
            image_response = requests.get(svg_url, headers=HEADERS)
            image_response.raise_for_status()
            
            with open(chemin_fichier, 'wb') as f:
                f.write(image_response.content)
            
            drapeaux_telecharges += 1
            print(f"[OK] Téléchargé ({i+1}/{len(valid_file_page_links)}) : {nom_fichier}")
            
            time.sleep(0.5) 
            
        except requests.exceptions.RequestException as e:
            print(f"[ERREUR] Échec du traitement de {nom_fichier}: {e}")
            time.sleep(1) 
            
    # --- FIN DU LOOP ---
    print("\n-----------------------------------------")
    print(f"✅ Terminé. **{drapeaux_telecharges}** fichiers SVG de haute résolution téléchargés dans le dossier '{dossier_cible}'.")
    print("-----------------------------------------")

# --- EXÉCUTION ---
telecharger_drapeaux_svg_final_v3(URL_WIKIPEDIA, DOSSIER_CIBLE)


--- DÉBOGAGE ÉTAPE 1 : ACCÈS À LA PAGE PRINCIPALE ---
[INFO] 199 liens de pages de fichiers trouvés. Lancement du loop...
-----------------------------------------------------------------
[OK] Téléchargé (1/199) : 2017-fr.wp-orange-source.svg
[OK] Téléchargé (2/199) : Flags-1900.jpg
[OK] Téléchargé (3/199) : Flag_of_the_Taliban.svg
[OK] Téléchargé (4/199) : Flag_of_South_Africa.svg
[OK] Téléchargé (5/199) : Flag_of_Albania.svg
[OK] Téléchargé (6/199) : Flag_of_Algeria.svg
[OK] Téléchargé (7/199) : Flag_of_Germany.svg
[OK] Téléchargé (8/199) : Flag_of_Andorra.svg
[OK] Téléchargé (9/199) : Flag_of_Angola.svg
[OK] Téléchargé (10/199) : Flag_of_Antigua_and_Barbuda.svg
[OK] Téléchargé (11/199) : Flag_of_Saudi_Arabia.svg
[OK] Téléchargé (12/199) : Flag_of_Argentina.svg
[OK] Téléchargé (13/199) : Flag_of_Armenia.svg
[OK] Téléchargé (14/199) : Flag_of_Australia_(converted).svg
[OK] Téléchargé (15/199) : Flag_of_Austria.svg
[OK] Téléchargé (16/199) : Flag_of_Azerbaijan.svg
[OK] Téléchargé (17/

In [70]:
import os

DOSSIER_CIBLE = "drapeaux"

try:
    # Liste tous les fichiers et dossiers dans le répertoire cible
    liste_fichiers = os.listdir(DOSSIER_CIBLE)
    
    # Filtre pour n'afficher que les fichiers de drapeaux (SVG ou autres formats d'image)
    fichiers_drapeaux = [
        f for f in liste_fichiers 
        if f.lower().endswith(('.svg', '.png', '.jpg', '.jpeg'))
    ]

    print(f"\n--- Liste des {len(fichiers_drapeaux)} fichiers de drapeaux dans le dossier '{DOSSIER_CIBLE}' ---\n")
    
    # Affiche la liste proprement
    for i, nom_fichier in enumerate(fichiers_drapeaux):
        print(f"[{i+1}] {nom_fichier}")

except FileNotFoundError:
    print(f"❌ Erreur : Le dossier '{DOSSIER_CIBLE}' n'a pas été trouvé. Assurez-vous d'exécuter ce code au même endroit que le script de téléchargement.")


--- Liste des 197 fichiers de drapeaux dans le dossier 'drapeaux' ---

[1] Nigéria.svg
[2] Australie.svg
[3] Érythrée.svg
[4] Slovénie.svg
[5] République centrafricaine.svg
[6] Azerbaïdjan.svg
[7] Panama.svg
[8] Soudan.svg
[9] Oman.svg
[10] Senegal.svg
[11] Norvège.svg
[12] Madagascar.svg
[13] Géorgie.svg
[14] Tonga.svg
[15] Andorre.svg
[16] Gabon.svg
[17] Saint Vincent Et Les Grenadines.svg
[18] Syrie.svg
[19] Mozambique.svg
[20] Lituanie.svg
[21] Libye.svg
[22] Bahreïn.svg
[23] Maurice.svg
[24] Grenade.svg
[25] Maroc.svg
[26] Seychelles.svg
[27] Afrique du Sud.svg
[28] Burundi.svg
[29] Mexique.svg
[30] Biélorussie.svg
[31] Monaco.svg
[32] Slovaquie.svg
[33] Israël.svg
[34] Égypte.svg
[35] République démocratique du Congo (2).svg
[36] République dominicaine.svg
[37] Sainte Lucie.svg
[38] Niue.svg
[39] Kiribati.svg
[40] Nouvelle Zelande.svg
[41] Honduras.svg
[42] Bosnie-Herzégovine.svg
[43] Équateur.svg
[44] Macédoine du Nord.svg
[45] Russie.svg
[46] Turkménistan.svg
[47] Rwanda.svg
[